# IMPORTACION DE LAS LIBRERIAS 


Para trabajar con esta parte del proyecto es necesario importar algunas librerías para manejar mejor los datos proporcionados. En este notebook, se enfocará en explorar y analizar las relaciones existentes entre las variables: 'release_year', 'gender_processed', 'Country'. El objetivo principal es observar cómo el paso del tiempo (indicado por la variable 'release_year') afecta a los géneros (indicado por la variable 'gender_type'). Por otro lado, ver cuales son los géneros que mas triunfan en cada pais (indicado por la variable'Country')

In [4]:
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
import random 
import plotly.graph_objects as go
import matplotlib.colors as mcolors
import mpld3

# Analisis de Tiempo y Género.

Para llevar a cabo este análisis, se deben tener en cuenta varios pasos clave:

1) **Limpieza y Preparación de Datos:** Primero, se debe asegurar que el conjunto de datos esté limpio y listo para el análisis. Esto incluye eliminar las filas con valores nulos, cero o menores de 2000 en la variable 'release_year', ya que esto podría perjudicar el análisis, así como limpiar las variables 'gender_type' y 'Country' para que contengan datos válidos y no sucios.

2) **Encolumnar los Géneros:** A partir de la columna 'gender_type' y 'Country' , se deben agregar columnas al dataset para cada género y pais, indicando con 1 si la película pertenece y con 0 si no pertenece.

3) **Estudio de Decisión:** Se explicará el origen de las decisiones tomadas durante el proceso de análisis.

4) **Exploración y visualización de Datos:** Para complementar el análisis cuantitativo, se crearán visualizaciones como histogramas que permitan observar la relación entre el año y el género más repetido, de una forma más intuitiva. Estas visualizaciones pueden ayudar a identificar patrones, tendencias o agrupaciones específicas dentro de los datos.

6) **Interpretación y Conclusiones:** Finalmente, se interpretarán los resultados del análisis para extraer conclusiones sobre la relación entre el año de salida de una película y el género más repetido. Se considerará en qué año estrenar una película y qué género podría interesar producir. También se discutirán las limitaciones del análisis y posibles pasos futuros para una investigación más profunda.

# 0) Visualización de Datos.

Primero, se llamará y leerá el dataset 'df' para facilitar el trabajo. 

In [5]:
df = pd.read_csv("./all_streaming.csv")
df

,movie_or_serie,title,director,cast,country,date_added_platform,release_year,duration_seconds,gender_type,description,channel_streaming
0,Movie,ricky velez: here's everything,uninformed director,uninformed cast,uninformed country,"October 24, 2021",2021,NaN,"comedy, stand up",​Comedian Ricky Velez bares it all with his ho...,hulu-movies-and-tv-shows
1,Movie,silent night,uninformed director,uninformed cast,uninformed country,"October 23, 2021",2020,94 min,"crime, drama, thriller","Mark, a low end South London hitman recently r...",hulu-movies-and-tv-shows
2,Movie,the marksman,uninformed director,uninformed cast,uninformed country,"October 23, 2021",2021,108 min,"action, thriller",A hardened Arizona rancher tries to protect an...,hulu-movies-and-tv-shows
3,Movie,gaia,uninformed director,uninformed cast,uninformed country,"October 22, 2021",2021,97 min,horror,A forest ranger and two survivalists with a cu...,hulu-movies-and-tv-shows
4,Movie,settlers,uninformed director,uninformed cast,uninformed country,"October 22, 2021",2021,104 min,"science fiction, thriller",Mankind's earliest settlers on the Martian fro...,hulu-movies-and-tv-shows
...,...,...,...,...,...,...,...,...,...,...,...
22993,Movie,zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,158 min,"cult ,, dramas, thrillers","A political cartoonist, a crime reporter and a...",netflix-shows
22994,TV Show,zombie dumb,uninformed director,uninformed cast,uninformed country,"July 1, 2019",2018,2 Seasons,"kids' ,, korean ,, , comedies","While living alone in a spooky town, a young g...",netflix-shows
22995,Movie,zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,88 min,"comedies, horror ,",Looking to survive in a world taken over by zo...,netflix-shows
22996,Movie,zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,88 min,"children , family ,, comedies","Dragged from civilian life, a former superhero...",netflix-shows


# 1) Limpieza y Preparación de Datos.


A continuación, se procederá con el primer paso clave: limpiar la columna 'gender_type' y 'country', debido a que se encontraron muchos datos sucios.

1. **Espacios al principio:** Se corregirán los datos que tienen espacios al principio, lo cual ocurre frecuentemente por errores de escritura.

2. **Espacios al final:** Al igual que los espacios al principio, se limpiarán los datos con espacios al final.

3. **Fallos con 'and':** Los datos separados por 'and' se limpiarán para evitar interferencias durante la limpieza, ya que algunos datos combinan dos aspectos en uno solo.
 

In [6]:
generos_fila = df['gender_type']
aux = []
for fila in generos_fila:
    aux.append([gender[1:] if gender.startswith(" ") else gender for gender in fila])
generos_fila = aux
aux = []
for fila in generos_fila:
    aux.append([gender[:-1] if gender.endswith(" ") else gender for gender in fila])
generos_fila = aux 
aux = []
for fila in generos_fila:
    aux.append([gender[4:] if gender.startswith("and ") else gender for gender in fila])
generos_fila = aux
generos_fila_lim = [[elemento for elemento in sublista if elemento] for sublista in generos_fila]

In [7]:
paises_fila = df['country']
aux = []
for fila in paises_fila:
    aux.append([gender[1:] if gender.startswith(" ") else gender for gender in fila])
paises_fila = aux
aux = []
for fila in paises_fila:
    aux.append([gender[:-1] if gender.endswith(" ") else gender for gender in fila])
paises_fila = aux 
aux = []
for fila in paises_fila:
    aux.append([gender[4:] if gender.startswith("and ") else gender for gender in fila])
paises_fila = aux
paisess_fila_lim = [[elemento for elemento in sublista if elemento] for sublista in paises_fila]

Después de limpiar las columnas, se eliminarán las películas producidas antes del año 2000, ya que el estudio se enfoca en un periodo más actual y estos datos antiguos no son significativos. Luego, se identificarán y corregirán las variaciones en la escritura de los datos, ya sea por plurales, contracciones o caracteres adicionales irrelevantes.

In [8]:

df = df[df['release_year'] >= 2000]
df = df[df['release_year'].notnull() & (df['release_year'] != 0)]
generos_unicos = df['gender_type'].explode().unique()
generos_unicos = df['gender_type'].str.split(',').explode().str.strip().unique()

for i in range(len(generos_unicos)):
    if generos_unicos[i].startswith('and '):
        generos_unicos[i] = generos_unicos[i][4:]
    generos_unicos[i] = generos_unicos[i].replace("kids'", 'kids').replace('sci fi', 'science fiction').replace('dramas', 'drama').replace('lgbtq+', 'lgbtq').replace('anime features', 'anime').replace('anime series','anime').replace('comedy', 'comedies').replace('documentary', 'documentaries').replace('romance', 'romantic').replace('game shows', 'game show').replace('mysteries', 'mystery').replace('musicals', 'musical').replace('thrillers','thriller').replace('talk shows', 'talk show').replace('stand', 'stand up').replace('stand up','stand up comedy').replace('up comedies','stand up comedy up').replace('stand up comedy up','stand up comedy').replace('science','sci')
generos = list(set(generos_unicos))
generos=[generos_unicos for generos_unicos in generos if generos_unicos]
print(generos)


['competition', 'survival', 'animation', 'faith and spirituality', 'coming of age', 'parody', 'horror', 'suspense', 'comedies', 'melodrama', 'sports', 'arthouse', 'concert film', 'series', 'spy', 'sitcom', 'action', 'sci fiction', 'food', 'british', 'latino', 'police', 'mystery', 'news', 'travel', 'cooking', 'music', 'cop', 'independent', 'cartoons', 'spirituality', 'buddy', 'fantasy', 'late night', 'adult animation', 'biographical', 'technology', 'family', 'musical', 'anthology', 'kids', 'lgbtq', 'korean', 'cult', 'spanish', 'historical', 'special interest', 'health', 'international', 'romantic', 'military and war', 'adventure', 'arts', 'game show', 'entertainment', 'romantic comedies', 'music videos and concerts', 'animals', 'fitness', 'fi', 'soap opera', 'language', 'drama', 'culture', 'anime', 'disaster', 'talk show and variety', 'superhero', 'wellness', 'dance', 'stand up comedy', 'young adult audience', 'documentaries', 'classic', 'history', 'black stories', 'children', 'teen', '

In [9]:
paises_unicos = df['country'].explode().unique()
paises_unicos = df['country'].str.split(',').explode().str.strip().unique()
for i in range(len(paises_unicos)):
    if paises_unicos[i].startswith('and '):
        paises_unicos[i] = paises_unicos[i][4:]
paises = list(set(paises_unicos))
paises=[paises_unicos for paises_unicos in paises if paises_unicos]
print(paises)

['South Korea', 'Switzerland', 'New Zealand', 'Mongolia', 'Japan', 'Morocco', 'Kuwait', 'Namibia', 'Slovakia', 'Ukraine', 'Ghana', 'Paraguay', 'Mauritius', 'Syria', 'Uganda', 'Poland', 'Jordan', 'Sweden', 'Ireland', 'Luxembourg', 'Colombia', 'Zimbabwe', 'Australia', 'Azerbaijan', 'Samoa', 'Lithuania', 'Russia', 'Indonesia', 'United Arab Emirates', 'Israel', 'Cuba', 'Georgia', 'Mexico', 'Turkey', 'Chile', 'Montenegro', 'France', 'Nigeria', 'Netherlands', 'Romania', 'Cambodia', 'Argentina', 'Spain', 'Latvia', 'Vatican City', 'Saudi Arabia', 'Norway', 'Iceland', 'Slovenia', 'Cyprus', 'Costa Rica', 'Algeria', 'Puerto Rico', 'Czech Republic', 'Guatemala', 'Croatia', 'Malta', 'Malawi', 'Monaco', 'Canada', 'Dominican Republic', 'Singapore', 'Uruguay', 'Venezuela', 'Pakistan', 'Egypt', 'Jamaica', 'Taiwan', 'Bangladesh', 'Belarus', 'Kazakhstan', 'China', 'Lebanon', 'Italy', 'Portugal', 'Panama', 'Belgium', 'Denmark', 'South Africa', 'Iraq', 'United Kingdom', 'Greece', 'Malaysia', 'Bermuda', 'In

# 2) Encolumnar los géneros

Una vez limpias las listas de géneros, se compararán con las variables 'gender_type'y 'country', utilizando un umbral de similitud entre palabras para sobrescribir los datos sucios con los limpios. Posteriormente, se crearán las columnas de los datos con sus correspondientes 0 y 1 según si la película pertenece o no a las columnas creadas.

In [10]:
umbral_similitud = 85
for genero in generos:
    df[genero] = df['gender_type'].apply(lambda x: 1 if fuzz.token_set_ratio(genero, str(x)) >= umbral_similitud else 0)

In [ ]:
umbral_similitud = 85
for pais in paises:
    df[pais] = df['country'].apply(lambda x: 1 if fuzz.token_set_ratio(pais, str(x)) >= umbral_similitud else 0)


# 3) Estudio de decisión

Las decisiones tomadas para esta parte del trabajo se han consultado en estudios realizados por expertos. Estos estudios han servido para confirmar las acciones:

1) **Umbral de similitud semantica:** Se ha definido un umbral de similitud promedio basado en las medidas de Path, Wu y Palmer, y Li, logrando un 92% de exactitud para este tipo de relaciones semánticas, con un __85%__ de concordancia con la validación de un experto. __(Benemérita Universidad Autónoma de Puebla, Facultad de Ciencias de la Computación,Puebla, México)__.

      [enlace en línea](https://rcs.cic.ipn.mx/2018_147_6/Medidas%20de%20similitud%20semantica%20aplicadas%20a%20una%20ontologia%20de%20dominio.pdf)

2) **Limitación de años:** La limitación a partir del año 2000 se debe a que el acceso a contenido multimedia en línea se facilitó significativamente a partir de entonces, cambiando la dinámica de consumo de películas .__(Universidad laica vicente rocafuerte, de guayaquil, facultad de ciencias sociales y derecho, carrera de periodismo, tema, plataformas streaming, autor, byron walther vélez albán, Guayaquil, 2019)__.

     [enlace en línea](http://repositorio.ulvr.edu.ec/bitstream/44000/3068/1/T-ULVR-2711.pdf)

3) **Eliminacion de Géneros:** Se ha decidido eliminar géneros no representados en al menos la mitad de los años estudiados, basándose en la gráfica de barras que muestra la representatividad de los géneros en los años seleccionados __('grafica_de_barras')__ , como lo aprendido en la asignatura **MET** de este mismo curso 




# 4) Exploración y Visualización de Datos

### Análisis de la Relación entre Géneros de Películas por Año


Tras estudiar las gráficas, se ha observado que muchos géneros no son relevantes, ya que no están presentes en al menos 10 años (comprobado estudiando las graficas de la función 'grafica_de_barras'). Se ha optado por utilizar este criterio de los 10 años, considerando que un género debe estar representado en más de la mitad de los años estudiados para ser significativo en el análisis. 

In [12]:
def grafica_de_barras(df):
    for columna in df.columns:
        if columna != 'release_year':
            plt.figure(figsize=(10, 6))
            peliculas_por_año = df[df[columna] == 1].groupby('release_year').size()
            plt.bar(peliculas_por_año.index.astype(int), peliculas_por_año.values)
            plt.title(f'Número de películas de {columna} por año')
            plt.xlabel('Año')
            plt.ylabel('Número de Películas')
            plt.grid(True)
            plt.tight_layout()
            plt.show()




Se han modificado algunas variables para crear una gráfica interactiva que muestra la evolución de la producción de los géneros de películas a lo largo de los años, siguiendo estos pasos.

1. **Convertir Datos en un Formato Más Largo:** Se realiza una transformación en los datos para cambiar su estructura original, pasando de tener múltiples columnas de géneros a un formato más largo donde cada fila representa un género de una película específica. Esto simplifica el análisis de los datos.

2. **Eliminar Datos Incorrectos o Faltantes:** Se procede a limpiar los datos para eliminar cualquier información incorrecta o faltante que pueda afectar la precisión del análisis.

3. **Agrupar y Contar:** Se agrupan las películas según su año de lanzamiento y tipo de género, contando cuántas películas hay para cada combinación de año y género. Esto proporciona una visión general de la distribución de géneros a lo largo del tiempo.

4. **Calcular Totales:** Se suman las cantidades de películas por género para cada año, lo que nos da el total de películas lanzadas en cada año.

5. **Calcular Porcentajes:** Utilizando la información de la cantidad de películas por género y los totales de películas por año, se calcula el porcentaje de películas de cada género en relación con el total. Esto permite comprender la proporción de cada tipo de película en el conjunto de datos.


In [13]:
df_long = df.melt(id_vars='release_year', value_vars=generos, var_name='gender_type', value_name='count')
df_long['count'] = pd.to_numeric(df_long['count'], errors='coerce')
df_long = df_long[df_long['count'] > 0]
df_grouped = df_long.groupby(['release_year', 'gender_type']).size().reset_index(name='counts')
df_total = df_grouped.groupby('release_year')['counts'].sum().reset_index(name='total_counts')
df_merged = pd.merge(df_grouped, df_total, on='release_year')
df_merged['percentage'] = (df_merged['counts'] / df_merged['total_counts']) * 100



El siguiente gráfico de barras ilustra la evolución de los géneros de películas a lo largo de los años. Observamos lo siguiente:

1. **Distribución de los datos:** Los géneros de películas varía considerablemente de un año a otro. Algunos géneros mantienen una popularidad constante a lo largo del tiempo, mientras que otros experimentan fluctuaciones significativas.

2. **Tendencias visibles:** A lo largo de los años, algunos géneros muestran una tendencia clara de aumento o disminución en producción, lo que puede estar influenciado por factores como las tendencias culturales, los éxitos de taquilla y los cambios en las preferencias del público.

3. **Interpretación de colores:** En el gráfico, los géneros se representan con colores específicos, lo que permite una fácil identificación y comparación visual entre ellos. Se ha utilizado un esquema de colores aleatorios para resaltar cada género de manera única.

4. **Análisis de tendencias:** Se observa una amplia variedad de tendencias entre los géneros de películas, lo que sugiere que la producción de un género puede verse afectada por una combinación de factores externos e internos a lo largo del tiempo.

Este análisis proporciona una visión general de cómo los géneros de películas han evolucionado a lo largo de los años, lo que puede ser útil para comprender las preferencias del público y tomar decisiones relacionadas con la producción y distribución de contenido cinematográfico.


In [14]:
top_genres = df_merged.groupby('gender_type')['counts'].sum().index.tolist()

# Creamos un diccionario de colores fijos para cada género
color_map = {genre: "#{:06x}".format(random.randint(0, 0xFFFFFF)) for genre in top_genres}

# Creación de los frames para la animación
frames = []
for year in df_merged['release_year'].unique():
    df_year = df_merged[df_merged['release_year'] == year].sort_values(by='percentage', ascending=False)
    
    # Ordenar los géneros de mayor a menor porcentaje para el año actual
    sorted_genres = df_year['gender_type'].tolist()
    
    data = []
    for genre in sorted_genres:
        percentage = df_year[df_year['gender_type'] == genre]['percentage'].iloc[0]
        color = color_map[genre]
        data.append(go.Bar(x=[genre], y=[percentage], marker_color=color, name=genre))
    
    # Encontrar el género con el mayor porcentaje en el año actual
    max_genre = df_year.iloc[0]['gender_type'] if not df_year.empty else 'N/A'
    
    frame = go.Frame(
        data=data,
        name=str(year),
        layout=go.Layout(
            annotations=[
                go.layout.Annotation(
                    text=f"Año: {year}<br>Género más popular: {max_genre}",
                    xref='paper', yref='paper',
                    x=0, y=1, showarrow=False,
                    font=dict(size=30)
                )
            ]
        )
    )
    frames.append(frame)

# Datos y layout del gráfico inicial
initial_year = df_merged['release_year'].min()
df_initial = df_merged[df_merged['release_year'] == initial_year].sort_values(by='percentage', ascending=False)

# Ordenar los géneros de mayor a menor porcentaje para el año inicial
sorted_genres_initial = df_initial['gender_type'].tolist()

data_initial = []
for genre in sorted_genres_initial:
    percentage = df_initial[df_initial['gender_type'] == genre]['percentage'].iloc[0]
    color = color_map[genre]
    data_initial.append(go.Bar(x=[genre], y=[percentage], marker_color=color, name=genre))

layout = go.Layout(
    xaxis=dict(title='Género'),
    yaxis=dict(title='Porcentaje', range=[0, 20]),  # Ajustar el rango si es necesario
    title='Evolución de la popularidad de los géneros de películas',
    annotations=[
        go.layout.Annotation(
            text=f"Año: {initial_year}<br>Género más popular: {df_initial.iloc[0]['gender_type'] if not df_initial.empty else 'N/A'}",
            xref='paper', yref='paper',
            x=0, y=1, showarrow=False,
            font=dict(size=30)
        )
    ],
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {
                    'frame': {'duration': 5000, 'redraw': True},  # Duración de 5000 ms (5 segundos) por frame
                    'fromcurrent': True,
                    'transition': {'duration': 500, 'easing': 'linear'}
                }],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

# Crear el gráfico inicial con los colores definidos en el diccionario
fig = go.Figure(
    data=data_initial,
    layout=layout,
    frames=frames
)

# Añadir la leyenda
fig.update_layout(showlegend=True)

# Guardar el gráfico en un archivo HTML
fig.write_html('evolucion_generos.html')


Este análisis proporciona una visión dinámica de cómo la producción de los géneros cinematográficos ha evolucionado a lo largo de los años. Al asignar colores según las variaciones en el porcentaje de películas de cada género, el gráfico destaca de manera efectiva los cambios significativos en la preferencia de los creadores a lo largo del tiempo. La codificación de colores facilita la identificación de tendencias ascendentes, descendentes o estables en la popularidad de cada género, lo que puede ser valioso para comprender mejor las preferencias del público y orientar estrategias de producción y distribución cinematográfica.

In [15]:
top_genres = df_merged.groupby('gender_type')['counts'].sum().nlargest(5).index.tolist()

# Crear un diccionario de colores iniciales para cada género
color_map = {genre: 'blue' for genre in top_genres}

# Crear los frames para la animación
frames = []
previous_data = None
for year in df_merged['release_year'].unique():
    df_year = df_merged[df_merged['release_year'] == year].sort_values(by='percentage', ascending=False)
    
    data = []
    for genre in top_genres:
        percentage = df_year[df_year['gender_type'] == genre]['percentage'].iloc[0] if genre in df_year['gender_type'].values else 0
        color = color_map[genre]
        data.append(go.Bar(x=[genre], y=[percentage], marker_color=color, name=genre))
    
    # Comparar con el año anterior y asignar colores según la variación
    if previous_data is not None:
        for i, genre in enumerate(top_genres):
            previous_percentage = previous_data[i]['y'][0]
            current_percentage = data[i]['y'][0]
            if current_percentage > previous_percentage * 1.1:
                data[i]['marker']['color'] = 'green'  # Subió más del 10%, color verde
            elif current_percentage < previous_percentage * 0.9:
                data[i]['marker']['color'] = 'red'  # Bajó más del 10%, color rojo
            else:
                data[i]['marker']['color'] = 'yellow'  # Se mantuvo, color amarillo
    
    frame = go.Frame(
        data=data,
        name=str(year),
        layout=go.Layout(
            annotations=[
                go.layout.Annotation(
                    text=str(year),
                    xref='paper', yref='paper',
                    x=0, y=1, showarrow=False,
                    font=dict(size=30)
                )
            ]
        )
    )
    frames.append(frame)
    
    previous_data = data

# Datos y layout del gráfico inicial
initial_year = df_merged['release_year'].min()
df_initial = df_merged[df_merged['release_year'] == initial_year].sort_values(by='percentage', ascending=False)

data_initial = []
for genre in top_genres:
    percentage = df_initial[df_initial['gender_type'] == genre]['percentage'].iloc[0] if genre in df_initial['gender_type'].values else 0
    color = color_map[genre]
    data_initial.append(go.Bar(x=[genre], y=[percentage], marker_color=color, name=genre))

layout = go.Layout(
    xaxis=dict(title='Género'),
    yaxis=dict(title='Porcentaje', range=[0, 20]),  # Ajustar el rango si es necesario
    title='Evolución de la popularidad de los géneros de películas',
    annotations=[
        go.layout.Annotation(
            text=str(initial_year),
            xref='paper', yref='paper',
            x=0, y=1, showarrow=False,
            font=dict(size=30)
        )
    ],
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {
                    'frame': {'duration': 10000, 'redraw': True},  # Duración de 10000 ms (10 segundos) por frame
                    'fromcurrent': True,
                    'transition': {'duration': 500, 'easing': 'linear'}
                }],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

# Crear el gráfico inicial con los colores definidos en el diccionario
fig = go.Figure(
    data=data_initial,
    layout=layout,
    frames=frames
)

# Añadir la leyenda
fig.update_layout(showlegend=True)

# Guardar el gráfico en un archivo HTML
fig.write_html('evolucion_generos_semaforo.html')

# Mensaje para abrir el archivo
print("El gráfico ha sido guardado como 'evolucion_generos_semaforo.html'. Ábrelo en tu navegador para verlo.")

El gráfico ha sido guardado como 'evolucion_generos_semaforo.html'. Ábrelo en tu navegador para verlo.


Este código realiza un análisis detallado de la distribución de géneros cinematográficos a lo largo de los años. Primero, se utiliza la función __pd.pivot_table__ para calcular la cantidad total de películas de cada género para cada año. Luego, se convierten los valores de la tabla pivotante en números para poder realizar cálculos numéricos.

A continuación, se calcula el porcentaje de cada género con respecto al total de películas producidas en cada año. Esto se logra dividiendo cada valor en la tabla pivotante por el total de películas producidas ese año y multiplicando por 100. El resultado es una tabla que muestra el porcentaje de películas de cada género para cada año.

Después, se filtran los géneros que se desean analizar más detalladamente, probablemente los más relevantes o interesantes para el estudio en cuestión.

El siguiente paso implica dividir los géneros en grupos de hasta 16 para evitar la sobrecarga visual en el gráfico. Luego, se itera sobre cada grupo y se crea un gráfico de líneas para cada género, mostrando cómo varía su porcentaje a lo largo de los años.

Finalmente, se muestra el gráfico, asegurándose de que esté bien formateado y legible para el usuario. Este proceso se repite para cada grupo de géneros, lo que permite una exploración exhaustiva de la distribución de géneros a lo largo del tiempo.


In [16]:
pivot_table = pd.pivot_table(df, index='release_year', aggfunc='sum')
pivot_table = pivot_table.apply(pd.to_numeric, errors='coerce')
total_por_año = pivot_table.sum(axis=1)
porcentaje_por_año = pivot_table.div(total_por_año, axis=0) * 100
porcentaje_filtrado = porcentaje_por_año[generos]

# Dividir los géneros en grupos de hasta 16 para evitar la sobrecarga visual
grupos_generos = [porcentaje_filtrado.columns[i:i+16] for i in range(0, len(porcentaje_filtrado.columns), 16)]

# Crear y guardar las gráficas en un archivo HTML
with open('grafica_porcentajes_generos.html', 'w') as f:
    for grupo in grupos_generos:
        fig, ax = plt.subplots(figsize=(12, 6))
        for column in grupo:
            ax.plot(porcentaje_filtrado.index, porcentaje_filtrado[column], marker='o', label=column)
        
        ax.set_title('Porcentaje de veces que aparecen los géneros por año')
        ax.set_xlabel('Año')
        ax.set_ylabel('Porcentaje de apariciones')
        ax.grid(True)
        ax.set_ylim([0, 20])
        ax.legend(loc='upper right', bbox_to_anchor=(1.15, 1), fontsize='small')
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # Convertir la figura a HTML y escribirla en el archivo
        html_str = mpld3.fig_to_html(fig)
        f.write(html_str)
        plt.close(fig)

print("Las gráficas han sido guardadas en 'grafica_porcentajes_generos.html'. Ábrelo en tu navegador para verlo.")


Las gráficas han sido guardadas en 'grafica_porcentajes_generos.html'. Ábrelo en tu navegador para verlo.



### Análisis Geoespacial de Preferencias de Contenido Audiovisual


El código proporcionado realiza un análisis geoespacial de las preferencias de contenido audiovisual a nivel mundial. A continuación, se explica paso a paso lo que hace cada parte del código:


1. **Inicialización de Variables:** Se crea un diccionario llamado __genero_por_pais__ que almacena el recuento de cada género de contenido por país, inicializándolo con valores de cero para cada género.


2. **Procesamiento del DataFrame:** Se recorre el DataFrame para contar los géneros de contenido audiovisual por país. Para cada fila del DataFrame, se verifica si el país tiene una marca (valor 1) y, si es así, se incrementa en uno el conteo del género correspondiente para ese país.


3. **Determinación del Género Más Común por País:** Se determina el género más común para cada país, seleccionando aquel con el recuento más alto.


4. **Creación del DataFrame de Géneros Máximos por País:** Se crea un DataFrame que contiene el país y su género de contenido más común.


5. **Mapeo de Nombres de Países:** Se aplica un mapeo de nombres de países para asegurar la coherencia con el conjunto de datos geoespaciales.


6. **Generación de Colores Aleatorios:** Se generan colores aleatorios para cada género de contenido.


7. **Añadir Colores al DataFrame:** Se añade una columna de colores al DataFrame de género máximo por país, asignando un color correspondiente a cada género.


8. **Lectura del Archivo Shapefile:** Se lee un archivo shapefile que contiene datos geoespaciales del mundo.


9. **Colorear los Países en el Mapa:** Se colorean los países en el mapa utilizando los colores asignados a los géneros máximos por país.


10. **Creación de una Leyenda:** Se crea una leyenda que muestra los géneros de contenido audiovisual y sus colores correspondientes.


En resumen, este código procesa el DataFrame que contiene información sobre la presencia de diferentes géneros de contenido audiovisual en cada país, y lo visualiza en un mapa del mundo coloreando cada país según su género de contenido más común y mostrando una leyenda para interpretar los colores.




In [ ]:

genero_por_pais = {pais: {genero: 0 for genero in generos} for pais in paises}

# Procesar el DataFrame para contar los géneros
for i, row in df.iterrows():
    for pais in paises:
        if row[pais] == 1:
            for genero in generos:
                if row[genero] == 1:
                    genero_por_pais[pais][genero] += 1

# Determinar el género más común para cada país
genero_maximo_por_pais = {}
for pais, conteo_generos in genero_por_pais.items():
    genero_maximo = max(conteo_generos, key=conteo_generos.get)
    genero_maximo_por_pais[pais] = genero_maximo

# Crear un DataFrame con los géneros máximos por país
df_genero_maximo = pd.DataFrame(list(genero_maximo_por_pais.items()), columns=['pais', 'genero_maximo'])

# Mapeo de nombres de países en el dataset geoespacial
mapeo_paises = {
    'United States': 'United States of America',
    'Dominican Republic': 'Dominican Rep.',
    'Czech Republic': 'Czechia',
}

# Aplicar el mapeo de nombres
df_genero_maximo['pais'] = df_genero_maximo['pais'].replace(mapeo_paises)

# Generar colores aleatorios para cada género
random.seed(42)  # Para reproducibilidad
colores = {genero: "#{:06x}".format(random.randint(0, 0xFFFFFF)) for genero in generos}

# Añadir una columna de colores al DataFrame del género máximo
df_genero_maximo['color'] = df_genero_maximo['genero_maximo'].map(colores)

# Leer el archivo shapefile para los datos geoespaciales
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Colorear los países sin información en gris
world = world.merge(df_genero_maximo, how='left', left_on='name', right_on='pais')
world['color'] = world['color'].fillna('lightgrey')

# Filtrar los géneros y colores que se han pintado
generos_pintados = df_genero_maximo['genero_maximo'].unique()
colores_pintados = {genero: colores[genero] for genero in generos_pintados}

# Plotear el mapa y guardar en un archivo HTML
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world.boundary.plot(ax=ax)
world.plot(ax=ax, color=world['color'])

# Crear una leyenda solo con los géneros pintados
handles = []
labels = []
for genero, color in colores_pintados.items():
    handles.append(plt.Line2D([], [], marker='o', color='w', markerfacecolor=color, markersize=10))
    labels.append(genero)

plt.legend(handles, labels, loc='upper left', title='Géneros')

plt.title('Mapa del Mundo Coloreado por Género Máximo de Contenido por País')

# Guardar la figura como HTML
html_str = mpld3.fig_to_html(fig)
with open('mapa_genero_maximo_por_pais.html', 'w') as f:
    f.write(html_str)

# Cerrar la figura para que no se muestre
plt.close(fig)

# Mostrar el mensaje de confirmación
print("El mapa ha sido guardado en 'mapa_genero_maximo_por_pais.html'. Ábrelo en tu navegador para verlo.")





# 5) Interpretación y Conclusiones

### Interpretación 

El análisis realizado ha revelado varios patrones y tendencias en la producción de géneros cinematográficos a lo largo del tiempo y su distribución geográfica. Al estudiar las gráficas y realizar una limpieza de datos para eliminar géneros con poca relevancia, se ha observado lo siguiente:

1. **Distribución Temporal de Géneros**: Algunos géneros cinematográficos han mantenido una producción constante a lo largo de los años, mientras que otros han mostrado fluctuaciones significativas. Esto sugiere que las tendencias culturales, los éxitos de taquilla y los cambios en las preferencias del público influyen en la producción de ciertos géneros.

2. **Tendencias Ascendentes y Descendentes**: Se han identificado géneros que han aumentado o disminuido en producción con el tiempo. Estos cambios pueden estar vinculados a diversos factores, como avances tecnológicos, cambios en la industria del entretenimiento y eventos socioeconómicos.

3. **Visualización Geoespacial**: El análisis geoespacial ha permitido identificar los géneros más populares en diferentes regiones del mundo. Al asignar colores específicos a cada género y visualizar esta información en un mapa, se puede observar cómo varían las preferencias de contenido audiovisual entre países.



### Conclusión

El análisis de la evolución temporal y geográfica de los géneros cinematográficos proporciona una comprensión profunda de cómo las preferencias del público han cambiado a lo largo del tiempo y entre diferentes regiones. Las principales conclusiones son:

- **Relevancia de Géneros**: La eliminación de géneros poco representados ha permitido centrarse en aquellos que tienen una presencia significativa en el tiempo, ofreciendo un análisis más preciso y relevante.

- **Identificación de Tendencias**: Las tendencias observadas en la producción de géneros pueden servir como indicadores de futuras preferencias y ayudar a predecir qué tipos de contenido podrían tener éxito.

- **Preferencias Regionales**: La visualización geoespacial ha destacado diferencias claras en las preferencias de contenido entre países, lo que es crucial para las estrategias de distribución y marketing en la industria cinematográfica.


Como se ha visto, este análisis proporciona herramientas valiosas para entender mejor el panorama de la industria cinematográfica y adaptar las estrategias de producción y distribución a las tendencias y preferencias emergentes, tanto a nivel temporal como geográfico.

En resumen, como ya se ha mencionado, los **análisis revelan patrones interesantes en la evolución de los géneros cinematográficos a lo largo de los años y sus preferencias geográficas**. Sin embargo, es importante señalar que las correlaciones encontradas no necesariamente implican una causalidad directa. Aunque se observan tendencias en la producción de ciertos géneros, no podemos afirmar con certeza si las productoras están adoptando estrategias específicas basadas en estas tendencias de audiencia.

Además, se destaca que un **menor número de películas de un género parece influir positivamente en el resultado global del mismo**. Esta observación podría atribuirse al hecho de que con menos películas en un género específico, existe menos dispersión de datos, lo que potencialmente lleva a una **mayor calidad promedio de las películas exitosas dentro de ese género**.

Desde esta perspectiva, las productoras podrían considerar **invertir en películas de géneros menos populares o de nicho**, ya que estas películas podrían destacarse más y convertirse en representantes destacados de su género. Esto es particularmente relevante en el contexto actual, con la **proliferación de plataformas de streaming que permiten una distribución amplia y accesible de una amplia gama de géneros cinematográficos**.
